In [1]:
# https://gitlab.com/haiqu-ai/qml-transpiler/-/issues/5

In [2]:
import qiskit

from qiskit.circuit.library import QuantumVolume

%cd -q ../..

from rivet_transpiler import get_circuit_hash

In [3]:
qv_circuit_1 = QuantumVolume(num_qubits=2, depth=1, seed=1)

qv_circuit_1.draw()

┌─────────────────────────┐
q_0: ┤0                        ├
     │  quantum_volume_[2,1,1] │
q_1: ┤1                        ├
     └─────────────────────────┘

In [4]:
qv_circuit_1.decompose(reps=2).draw(fold=-1)

global phase: 4.7181
     ┌──────────────────────────┐         ┌──────────────────────┐            ┌──────────────────┐         ┌───────────────────────────┐
q_0: ┤ U(1.0024,0.97963,2.9372) ├──■──────┤ U(0.50968,-π/2,-π/2) ├─────■──────┤ U(0.12562,0,π/2) ├──────■──┤ U(1.8645,-1.2663,-1.8282) ├
     ├──────────────────────────┤┌─┴─┐┌───┴──────────────────────┴──┐┌─┴─┐┌───┴──────────────────┴───┐┌─┴─┐└┬──────────────────────────┤
q_1: ┤ U(0.82902,2.3378,1.7888) ├┤ X ├┤ U(1.3313,-0.32827,-0.60827) ├┤ X ├┤ U(1.9404,-2.9753,2.0057) ├┤ X ├─┤ U(2.6566,-2.4584,1.0197) ├
     └──────────────────────────┘└───┘└─────────────────────────────┘└───┘└──────────────────────────┘└───┘ └──────────────────────────┘

In [5]:
qv_circuit_2 = QuantumVolume(num_qubits=2, depth=1, seed=2)

qv_circuit_2.draw()

┌─────────────────────────┐
q_0: ┤0                        ├
     │  quantum_volume_[2,1,2] │
q_1: ┤1                        ├
     └─────────────────────────┘

In [6]:
qv_circuit_2.decompose(reps=2).draw(fold=-1)

global phase: 4.242
     ┌───────────────────────────┐┌───┐┌─────────────────────────────┐┌───┐┌──────────────────────────┐┌───┐  ┌─────────────────────────┐  
q_0: ┤ U(0.42625,2.0777,-1.3067) ├┤ X ├┤ U(1.4041,-0.36948,-0.40469) ├┤ X ├┤ U(1.9404,-2.9753,2.0057) ├┤ X ├──┤ U(2.6464,3.0558,2.6067) ├──
     └┬──────────────────────────┤└─┬─┘└───┬──────────────────────┬──┘└─┬─┘└──┬───────────────────┬───┘└─┬─┘┌─┴─────────────────────────┴─┐
q_1: ─┤ U(0.22194,1.095,-2.3561) ├──■──────┤ U(0.90037,-π/2,-π/2) ├─────■─────┤ U(0.085471,0,π/2) ├──────■──┤ U(2.3676,-0.30944,-0.64982) ├
      └──────────────────────────┘         └──────────────────────┘           └───────────────────┘         └─────────────────────────────┘

In [40]:
import hashlib

def get_circuit_hash(circuit):

    hash_object = hashlib.sha256(b'')

    dag = qiskit.converters.circuit_to_dag(circuit)

    for node in dag.topological_op_nodes():

        operation = node.op

        qubit_indices = [circuit.find_bit(qubit).index for qubit in node.qargs]

        # Values

        values = []

        values.append(qubit_indices)
        values.append(operation.__class__)
        values.append(operation.num_qubits)
        values.append(operation.num_clbits)
        
        # print("qubit_indices:", qubit_indices)
        # print("operation.__class__:", operation.__class__)
        # print("operation.num_qubits:", operation.num_qubits)
        # print("operation.num_clbits:", operation.num_clbits)
        

        # Skip Parameters

        for parameter in operation.params:

            if isinstance(parameter, qiskit.circuit.parameter.ParameterExpression):

                parameter = None

            values.append(parameter)
            
            # print("parameter:", parameter)

        # Update Hash

        for value in values:

            encoded_value = repr(value).encode('utf-8')

            hash_object.update(encoded_value)

    hash_bytes = hash_object.digest()

    hash_value = int.from_bytes(hash_bytes, byteorder='little')

    return hash_value

In [7]:
get_circuit_hash(qv_circuit_1.decompose(reps=2))

81494582331337268712486896751651397882263172419496569560316079018007347265399

In [8]:
get_circuit_hash(qv_circuit_2.decompose(reps=2))

67621646157843688507862837171594541376831134984340120138614568120639168982402